In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Taxonomy Freq.csv")
df['label'] = df.index.values

In [3]:
df[['kingdom', 'phylum', 'class',
    'order', 'family', 'genus', 'species']] = df[['kingdom', 
                                                         'phylum', 'class', 'order', 'family', 'genus', 'species']].fillna(value="Unknown")

In [4]:
allLevels = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'end']
taxLevel = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'end']
df = df.drop((set(allLevels) - set(taxLevel)), 1)

def getDict(taxLevelIndex, prevIndex):
    list_dicts = []
    if taxLevel[taxLevelIndex] == 'end':
        return ([], 0)
    if taxLevel[taxLevelIndex] == 'kingdom':
        df_size = df.shape[0]
        gp = df.groupby(taxLevel[taxLevelIndex])
        
    else:
        df_size = df[df[taxLevel[taxLevelIndex-1]] == prevIndex].shape[0]
        gp = df[df[taxLevel[taxLevelIndex-1]] == prevIndex].groupby(taxLevel[taxLevelIndex])
        
    indexes = gp.size().index.to_list()
    values = list(gp.size().values)
    num_types = len(gp)
    ind_val = zip(indexes, values)
    ind_val.sort(key=lambda tup: tup[1], reverse = True)
    for i in ind_val:
        index = i[0]
        value = i[1]
        next_data, num_types_next = getDict(taxLevelIndex + 1, index)
        curr_gp = gp.get_group(index)
        redList = 0
        if (curr_gp.redList.unique().shape[0] > 1 or not pd.isna(curr_gp.redList.unique()[0])):
            redList = 1
        list_dicts.append({"index": curr_gp.iloc[0]['label'], "name": index, 
                       "taxLevel": taxLevel[taxLevelIndex], "types": num_types_next, 
                       "size": value, "ratio": (1.0 * value)/df_size, "children": next_data,
                       "redList": redList})
    
    
    return (list_dicts, num_types)
    
    #return a dictionary of name and children
    

In [ ]:
json_dict, num_types = getDict(0, "blah")
json_dict = {"index": - 1, "name": "Organisms", 'taxLevel': "organisms", "types": num_types, "children": json_dict, "ratio": 1.0,
            "size": df.shape[0], "redList": 1}


In [4]:
df = df[df.redList.notnull()]

In [6]:
allLevels = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'end']
taxLevel = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'end']
df = df.drop((set(allLevels) - set(taxLevel)), 1)

def getDict(taxLevelIndex, prevIndex):
    list_dicts = []
    if taxLevel[taxLevelIndex] == 'end':
        return ([], 0)
    if taxLevel[taxLevelIndex] == 'kingdom':
        df_size = df.shape[0]
        gp = df.groupby(taxLevel[taxLevelIndex])
        
    else:
        df_size = df[df[taxLevel[taxLevelIndex-1]] == prevIndex].shape[0]
        gp = df[df[taxLevel[taxLevelIndex-1]] == prevIndex].groupby(taxLevel[taxLevelIndex])
        
    indexes = gp.size().index.to_list()
    values = list(gp.size().values)
    num_types = len(gp)
    ind_val = zip(indexes, values)
    ind_val.sort(key=lambda tup: tup[1], reverse = True)
    for i in ind_val:
        index = i[0]
        value = i[1]
        next_data, num_types_next = getDict(taxLevelIndex + 1, index)
        curr_gp = gp.get_group(index)
        redList = 0
        if (curr_gp.redList.unique().shape[0] > 1 or not pd.isna(curr_gp.redList.unique()[0])):
            redList = 1
        if (taxLevel[taxLevelIndex] == "species"):
            list_dicts.append({"name": index, 
                           
                           "value": value, "children": next_data,
                           })
        else:
            list_dicts.append({"name": index, 
                           
                          "children": next_data,
                           })
    
    
    return (list_dicts, num_types)
    
    #return a dictionary of name and children
    

In [7]:
json_dict, num_types = getDict(0, "blah")
json_dict = {"name": "Organisms", "children": json_dict}


In [8]:
import json

with open('map_django/biodivmap/static/biodivmap/bar_chart.json', 'w') as fp:
    json.dump(json_dict, fp)

In [112]:
df_2.merge(df_1, how="outer")['class'].unique()

array(['Magnoliopsida', 'Pinopsida', 'Liliopsida', 'Polypodiopsida',
       'Marattiopsida', 'Psilotopsida', 'Cycadopsida', 'Lycopodiopsida',
       'Gnetopsida', 'Equisetopsida', 'Ginkgoopsida', 'Insecta'],
      dtype=object)